In [2]:
import numpy as np

%matplotlib notebook
import matplotlib.pyplot as plt

import sys
sys.path.append('..')
import utils as ut

DATA_PATH = '../../datav2/esp/'

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
df = ut.load_data(DATA_PATH,'train',nclasses=5).loc[:,['review_content','review_rate']]
df.head(5)

Dataset cargado para 5 clases (muy malo=0, malo=2, medio=3, bueno=4 muy bueno=1)
Num samples per category:
1    92477
2    92449
3    92487
4    92501
5    92454
Name: review_rate, dtype: int64


,review_content,review_rate
0,"Esta de muy baja calidad, no fue lo esperado.",2
1,"Reconozco que fue muy económico, pero su durab...",2
2,Muy bien muy bien muy bien muy bien muy bien m...,5
3,No me fue útil para mí. Diseño muy bueno.,2
4,No fue lo estipulado. Solo eso voy aclarar.,1


In [10]:
import csv

with open(DATA_PATH + 'train.csv', 'r') as f:
    csv_reader = csv.reader(f,delimiter=',',lineterminator='\n')
    fieldnames = next(csv_reader)
    for i, line in enumerate(csv_reader):
        if i >= 5:
            break
        print(line)

['country', 'category', 'review_content', 'review_title', 'review_rate']
['MLM', 'Salud, ropa y cuidado personal / Saúde, roupas e cuidado pessoal', 'Esta de muy baja calidad, no fue lo esperado.', 'No fue lo esperado ', '2']
['MLA', 'Salud, ropa y cuidado personal / Saúde, roupas e cuidado pessoal', 'Reconozco que fue muy económico, pero su durabilidad fue muy corta.', 'Malo', '2']
['MLV', 'Tecnología y electrónica / Tecnologia e electronica', 'Muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bien muy bi.', 'Excelente', '5']
['MLA', 'Salud, ropa y cuidado perso

In [3]:
from utils.tokenizers import RegexTokenizer, NLTKWordTokenizer, NLTKTweetTokenizer

def normalize_dataset(df):
    # Pasamos a minúscula todo
    df['review_content'] = df['review_content'].str.lower()
    # Sacamos todos los acentos
    for rep, rep_with in [('[óòÓöøôõ]','o'), ('[áàÁäåâãÄ]','a'), ('[íìÍïîÏ]','i'), 
                          ('[éèÉëêÈ]','e'), ('[úüÚùûÜ]','u'), ('[ç¢Ç]','c'), 
                          ('[ý¥]','y'),('š','s'),('ß','b'),('\x08','')]:
        df['review_content']  = df['review_content'].str.replace(rep,rep_with,regex=True)
    return df

def train_tokenizers(df_train,max_words,freq_cutoff,ngram_range,unk_token):

    tokenizers = {
        'regex': RegexTokenizer(r'(\w+|[\.,!\(\)"\-:\?/%;¡\$\'¿\\]|\d+)',
                                max_words,freq_cutoff,ngram_range,unk_token),
        'nltk': NLTKWordTokenizer('spanish',max_words,freq_cutoff,ngram_range,unk_token),
        'tweet': NLTKTweetTokenizer(max_words,freq_cutoff,ngram_range,unk_token)
    }


    for name, tknzr in tokenizers.items():
        print('{} tokenizer:'.format(name))
        tknzr.train(df_train['review_content'])
        print()
        
    return tokenizers
        

df = normalize_dataset(df)
df_train, df_dev = ut.train_dev_split(df,dev_size=0.1,random_state=2376482)

In [ ]:
from utils.classifiers import NaiveBayesClassifier
import pickle


max_words = [10000]
freq_cutoff = [1]
ngram_range = (1,2)
unk_token = None
reweight = ['none','tfidf','ppmi']

results = {}


for mw in max_words:
    for fc in freq_cutoff:
        tokenizers = train_tokenizers(df_train.iloc[:100],mw,fc,ngram_range,unk_token)
        for name,tknzr in tokenizers.items():
            for rw in reweight:
                # Fit:
                clf = NaiveBayesClassifier(alpha=1.0,num_features=mw,reweight=rw)
                ids_train = tknzr.sentences_to_ids(df_train.loc[:,'review_content'])
                labels_train = df_train.loc[:,'review_rate'].values
                clf.fit(ids_train,labels_train)
                # Predict:
                ids_dev = tknzr.sentences_to_ids(df_dev.loc[:,'review_content'])
                y = df_dev.loc[:,'review_rate'].values
                y_pred = clf.predict(ids_dev)
                print('Max words: {}, cutoff frequency: {}, reweight: {}'.format(mw,fc,rw))
                print('Accuracy: {:.2f}%'.format(sum(y_pred == y) * 100 / len(y_pred)))
                print()

                results[name] = {'max_words': mw, 'frequency_cutoff':fc, 'reweight':rw, 
                                 'ngram_range': ngram_range, 'unk_token': unk_token}
        pickle.dump(results,'./nb_classification_5_classes_{}_{}.pkl'.format(mw,fc))

regex tokenizer:
Vocab size: 4218

nltk tokenizer:
Vocab size: 4217

tweet tokenizer:
Vocab size: 4216

Max words: 10000, cutoff frequency: 1, reweight: none
Accuracy: 51.84%



In [12]:
y_pred.size, y.size

(416131, 46237)

In [2]:
df_train


NameError: name 'df_train' is not defined

In [17]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

clf = MultinomialNB()
clf.fit(X,y)
y_pred = clf.predict(X)
print(classification_report(y,y_pred))

              precision    recall  f1-score   support

         0.0       0.91      0.85      0.88    184750
         1.0       0.86      0.92      0.89    184750

    accuracy                           0.88    369500
   macro avg       0.89      0.88      0.88    369500
weighted avg       0.89      0.88      0.88    369500



In [18]:
clf.fit(X_tfidf,y)
y_pred = clf.predict(X_tfidf)
print(classification_report(y,y_pred))

              precision    recall  f1-score   support

         0.0       0.90      0.81      0.85    184750
         1.0       0.83      0.91      0.87    184750

    accuracy                           0.86    369500
   macro avg       0.87      0.86      0.86    369500
weighted avg       0.87      0.86      0.86    369500



In [19]:
clf.fit(X_ppmi,y)
y_pred = clf.predict(X_ppmi)
print(classification_report(y,y_pred))

              precision    recall  f1-score   support

         0.0       0.90      0.85      0.87    184750
         1.0       0.86      0.90      0.88    184750

    accuracy                           0.88    369500
   macro avg       0.88      0.88      0.88    369500
weighted avg       0.88      0.88      0.88    369500



In [2]:
import numpy as np

In [7]:
from scipy.sparse import csr_matrix

X = csr_matrix(np.array([[1,0,0,0,3,0,4],
                         [0,0,1,0,1,1,0],
                         [1,0,0,1,1,0,4],
                         [2,0,0,0,3,2,0],
                         [0,0,0,0,0,0,0]],dtype=float),shape=(5,7))

df = np.asarray(X.astype(bool,copy=True).sum(axis=0)).squeeze() / X.shape[0]
print(df)

np.bincount(X.indices, minlength=X.shape[1])/ X.shape[0]

[0.6 0.  0.2 0.2 0.8 0.4 0.4]


array([0.6, 0. , 0.2, 0.2, 0.8, 0.4, 0.4])